In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端ML：MLOps阶段2：实验：开始使用Vertex AI Training进行XGBoost

<table align="left">
    <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
    
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_xgboost.ipynb">
    <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了在谷歌云上使用Vertex AI进行端到端MLOps生产过程。本教程涵盖了阶段2：实验：开始使用Vertex AI训练XGBoost模型。

### 目标

在本教程中，您将学习如何使用 `Vertex AI Training` 来训练一个 XGBoost 自定义模型。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Training`
- `Vertex AI Model` 资源

执行的步骤包括：

- 使用 Python 软件包进行训练。
- 在超参数调整时报告准确性。
- 使用 GCSFuse 将模型工件保存到云存储中。
- 创建一个 `Vertex AI Model` 资源。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。这个数据集不需要任何特征工程。本教程中使用的数据集版本存储在一个公共的云存储桶中。训练好的模型可以预测三个种类中的一个：山鸢尾、维吉尼亚鸢尾或者变色鸢尾。

###成本

本教程使用 Google Cloud 的计费组件:

* Vertex AI
* Cloud Storage

了解有关 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和[Cloud Storage 价格](https://cloud.google.com/storage/pricing) 的信息，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用情况生成成本估算。

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q

### 重新启动内核

在安装了额外的软件包之后，您需要重新启动笔记本内核，以便它能够找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的谷歌云项目

**无论您使用的是什么笔记本环境都需要执行以下步骤。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter将以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目 ID

**如果您不知道您的项目 ID**，您可以使用 `gcloud` 获取您的项目 ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您也可以更改“REGION”变量，该变量在笔记本的其余部分中使用。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法在Vertex AI中使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}

if REGION == "[your-region]":
    REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源之间的名称冲突，您为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经经过验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行操作以通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **点击创建服务帐号**。

2. 在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

3. 在**授予这个服务帐号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex AI"，并选择**Vertex AI管理员**。在过滤框中输入"存储对象管理员"，并选择**存储对象管理员**。

4. 点击创建。一个包含您密钥的JSON文件将下载到您的本地环境。

5. 在下面的单元格中输入您的服务帐号密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时：运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对您的云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

设置变量

接下来，设置一些在教程中使用的变量。
导入库并定义常量。

In [ ]:
import google.cloud.aiplatform as aip

### 初始化 Vertex AI Python SDK

为您的项目和对应的存储桶初始化 Vertex AI Python SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个带有4个 Nvidia Telsa K80 GPU 的容器映像分配给每个 VM，您可以指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80, 4）

否则，指定`(None, None)`来使用一个用于在CPU上运行的容器映像。

了解有关您所在地区的[硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：用于 GPU 支持的 TF 2.3 之前的版本将无法在本教程中加载自定义模型。这是一个已知问题，在 TF 2.3 中已修复。这是由于在服务功能中生成的静态图操作引起的。如果在您自己的自定义模型上遇到此问题，请使用具有 GPU 支持的 TF 2.3 容器映像。

In [ ]:
import os

if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

将预先构建的Docker容器映像设置为用于训练和预测。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "xgboost-cpu.1-1"
DEPLOY_VERSION = "xgboost-cpu.1-1"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 设置变量`TRAIN_COMPUTE`来配置用于训练的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## XGBoost训练简介

一旦您训练了一个XGBoost模型，您会希望将其保存在云存储位置，以便随后上传到`Vertex AI Model`资源。 XGBoost软件包不支持将模型保存到云存储位置。相反，您将按照以下步骤保存到云存储位置。

1. 将内存中的模型保存到本地文件系统（例如，model.bst）。
2. 使用gsutil将本地副本复制到指定的云存储位置。

*注意*：您可以对XGBoost模型进行超参数调整。

### 检查培训包

#### 包布局

在开始培训之前，您将查看Python包是如何为自定义培训任务组装的。当解压缩后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是安装包到Docker镜像的操作环境的指令。

文件`trainer/task.py`是执行自定义培训任务的Python脚本。*注意*，当我们在工作池规范中引用它时，我们会用点(`trainer.task`)替换目录斜杠，并且去掉文件后缀(`.py`)。

#### 包装配

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'cloudml-hypertune',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Iris tabular classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 为Python训练包创建任务脚本

接下来，您可以为驱动训练包创建`task.py`脚本。一些值得注意的步骤包括：

- 命令行参数：
    - `model-dir`：保存训练模型的位置。在使用Vertex AI自定义训练时，位置将在环境变量中指定为`AIP_MODEL_DIR`，
    - `dataset_data_url`：要下载的训练数据的位置。
    - `dataset_labels_url`：要下载的训练标签的位置。
    - `boost-rounds`：可调参数
- 数据预处理（`get_data()`）：
    - 下载数据集并拆分为训练集和测试集。
- 训练（`train_model()`）：
    - 训练模型
- 评估（`evaluate_model()`）：
    - 评估模型。
    - 如果进行超参数调优，报告准确率指标。
- 模型工件保存
    - 将模型工件和评估指标保存在由`model-dir`指定的云存储位置。

In [ ]:
%%writefile custom/trainer/task.py
import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb
import hypertune
import argparse
import logging
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument("--dataset-data-url", dest="dataset_data_url",
                    type=str, help="Download url for the training data.")
parser.add_argument("--dataset-labels-url", dest="dataset_labels_url",
                    type=str, help="Download url for the training data labels.")
parser.add_argument("--boost-rounds", dest="boost_rounds",
                    default=20, type=int, help="Number of boosted rounds")
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

def get_data():
    logging.info("Downloading training data and labelsfrom: {}, {}".format(args.dataset_data_url, args.dataset_labels_url))
    # gsutil outputs everything to stderr so we need to divert it to stdout.
    subprocess.check_call(['gsutil', 'cp', args.dataset_data_url, 'data.csv'], stderr=sys.stdout)
    # gsutil outputs everything to stderr so we need to divert it to stdout.
    subprocess.check_call(['gsutil', 'cp', args.dataset_labels_url, 'labels.csv'], stderr=sys.stdout)


    # Load data into pandas, then use `.values` to get NumPy arrays
    data = pd.read_csv('data.csv').values
    labels = pd.read_csv('labels.csv').values

    # Convert one-column 2D array into 1D array for use with XGBoost
    labels = labels.reshape((labels.size,))

    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=7)

    # Load data into DMatrix object
    dtrain = xgb.DMatrix(train_data, label=train_labels)
    return dtrain, test_data, test_labels

def train_model(dtrain):
    logging.info("Start training ...")
    # Train XGBoost model
    params = {
        'objective': 'multi:softprob',
        'num_class': 3
    }
    model = xgb.train(params, dtrain, num_boost_round=args.boost_rounds)
    logging.info("Training completed")
    return model

def evaluate_model(model, test_data, test_labels):
    dtest = xgb.DMatrix(test_data)
    pred = model.predict(dtest)
    predictions = [np.around(value) for value in pred]
    # evaluate predictions
    try:
        accuracy = accuracy_score(test_labels, predictions)
    except:
        accuracy = 0.0
    logging.info(f"Evaluation completed with model accuracy: {accuracy}")

    # report metric for hyperparameter tuning
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='accuracy',
        metric_value=accuracy
    )
    return accuracy


dtrain, test_data, test_labels = get_data()
model = train_model(dtrain)
accuracy = evaluate_model(model, test_data, test_labels)

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

# Export the classifier to a file
gcs_model_path = os.path.join(args.model_dir, 'model.bst')
logging.info("Saving model artifacts to {}". format(gcs_model_path))
model.save_model(gcs_model_path)

logging.info("Saving metrics to {}/metrics.json". format(args.model_dir))
gcs_metrics_path = os.path.join(args.model_dir, 'metrics.json')
with open(gcs_metrics_path, "w") as f:
    f.write(f"{'accuracy: {accuracy}'}")

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_iris.tar.gz

### 创建并运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练任务，2) 运行这个任务。

#### 创建自定义训练任务

使用 `CustomTrainingJob` 类创建一个自定义训练任务，需要以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，以tarball形式。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

*注意：* 没有`requirements`参数。您可以在Python包的`setup.py`脚本中指定任何依赖。

In [ ]:
DISPLAY_NAME = "iris_" + UUID

job = aip.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_iris.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

### 准备您的命令行参数

现在为您自定义训练容器定义命令行参数：

- `args`：传递给设置为容器入口点的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用这个命令行参数来指定存储模型工件的位置。
      - 直接：您将云存储位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），或者
      - 间接：服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件的位置。
  - `--dataset-data-url`：要下载的训练数据的位置。
  - `--dataset-labels-url`：要下载的训练标签的位置。
  - `--boost-rounds`：可调节的超参数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, UUID)
DATASET_DIR = "gs://cloud-samples-data/ai-platform/iris"

ROUNDS = 20

DIRECT = False
if DIRECT:
    CMDARGS = [
        "--dataset-data-url=" + DATASET_DIR + "/iris_data.csv",
        "--dataset-labels-url=" + DATASET_DIR + "/iris_target.csv",
        "--boost-rounds=" + str(ROUNDS),
        "--model_dir=" + MODEL_DIR,
    ]
else:
    CMDARGS = [
        "--dataset-data-url=" + DATASET_DIR + "/iris_data.csv",
        "--dataset-labels-url=" + DATASET_DIR + "/iris_target.csv",
        "--boost-rounds=" + str(ROUNDS),
    ]

#### 运行自定义训练作业

接下来，您可以通过调用 `run` 方法来运行自定义作业，以启动训练作业，并使用以下参数：

- `model_display_name`：`Model` 资源的可读名称。
- `args`：传递到训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 是单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作器复制品的加速器数量。
- `base_output_dir`：将模型工件写入的 Cloud Storage 位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    model = job.run(
        model_display_name="iris_" + UUID,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=False,
    )
else:
    model = job.run(
        model_display_name="iris_" + UUID,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=False,
    )

model_path_to_deploy = MODEL_DIR

### 列出一个自定义培训任务

In [ ]:
_job = job.list(filter=f"display_name={DISPLAY_NAME}")
print(_job)

等待自定义训练作业完成

接下来，等待自定义训练作业完成。或者，可以在`run()`方法中将参数`sync`设置为`True`，以阻塞直到自定义训练作业完成。

In [ ]:
model.wait()

删除自定义训练作业

在训练作业完成后，您可以使用`delete()`方法删除训练作业。在完成之前，训练作业可以使用`cancel()`方法取消。

In [ ]:
job.delete()

# 清理工作

要清理此项目中使用的所有 Google Cloud 资源，您可以 [删除用于本教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的单个资源：

- 自定义作业（在上一步中删除了自定义训练作业）
- 云存储桶

In [ ]:
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI